In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
import warnings
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
import pyarrow.parquet as pq
from surprise import SVD
from surprise import accuracy
import pickle

In [2]:
games=pd.read_parquet("games.parquet")
#items=pd.read_parquet("items.parquet")
reviews=pd.read_parquet("reviews.parquet")

In [3]:
merged_df = pd.merge(reviews[['item_id', 'user_id', 'sentiment_analysis',"recommend"]], games[['app_name',"item_id"]], left_on='item_id', right_on="item_id")

In [59]:
merged_df

,item_id,user_id,sentiment_analysis,recommend,app_name,rating
0,1250,76561197970982479,2,1,Killing Floor,5.0
1,1250,death-hunter,2,1,Killing Floor,5.0
2,1250,DJKamBer,0,1,Killing Floor,1.0
3,1250,diego9031,1,1,Killing Floor,3.0
4,1250,76561198081962345,1,1,Killing Floor,3.0
...,...,...,...,...,...,...
44734,367780,laislabonita75,2,1,Aero's Quest,5.0
44735,367780,evilindiegaming,2,1,Aero's Quest,5.0
44736,305920,laislabonita75,0,0,Another Perspective,0.0
44737,306040,Gamer0009,2,1,The Howler,5.0


In [60]:
merged_df.shape

(44739, 6)

In [6]:
merged_df['recommend'] = merged_df['recommend'].replace({True: 1, False: 0})

In [7]:
for index, row in merged_df.iterrows():
    if (row[2] == 2) and (row[3] == 1):
        merged_df.at[index, 'rating'] = 5
    elif (row[2] == 2) and (row[3] == 0):
        merged_df.at[index, 'rating'] = 4
    elif (row[2] == 1) and (row[3] == 1):
        merged_df.at[index, 'rating'] = 3
    elif (row[2] == 1) and (row[3] == 0):
        merged_df.at[index, 'rating'] = 2
    elif (row[2] == 0) and (row[3] == 1):
        merged_df.at[index, 'rating'] = 1
    elif (row[2] == 0) and (row[3] == 0):
        merged_df.at[index, 'rating'] = 0

C:\Users\Vivobook\AppData\Local\Temp\ipykernel_16420\41572838.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (row[2] == 2) and (row[3] == 1):
C:\Users\Vivobook\AppData\Local\Temp\ipykernel_16420\41572838.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (row[2] == 2) and (row[3] == 1):
C:\Users\Vivobook\AppData\Local\Temp\ipykernel_16420\41572838.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif (row[2] == 2) and

In [8]:
merged_df

,item_id,user_id,sentiment_analysis,recommend,app_name,rating
0,1250,76561197970982479,2,1,Killing Floor,5.0
1,1250,death-hunter,2,1,Killing Floor,5.0
2,1250,DJKamBer,0,1,Killing Floor,1.0
3,1250,diego9031,1,1,Killing Floor,3.0
4,1250,76561198081962345,1,1,Killing Floor,3.0
...,...,...,...,...,...,...
44734,367780,laislabonita75,2,1,Aero's Quest,5.0
44735,367780,evilindiegaming,2,1,Aero's Quest,5.0
44736,305920,laislabonita75,0,0,Another Perspective,0.0
44737,306040,Gamer0009,2,1,The Howler,5.0


In [9]:
merged_df.to_parquet('modelo.parquet')

c:\Users\Vivobook\miniconda3\envs\datascience\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [10]:
# Leer el archivo .parquet y cargarlo en un DataFrame de pandas
new_df = pd.read_parquet('modelo.parquet')

In [44]:
df=pd.read_csv('modelo.csv')

In [45]:
df

,item_id,user_id,sentiment_analysis,recommend,app_name,rating
0,1250,76561197970982479,2,1,Killing Floor,5.0
1,1250,death-hunter,2,1,Killing Floor,5.0
2,1250,DJKamBer,0,1,Killing Floor,1.0
3,1250,diego9031,1,1,Killing Floor,3.0
4,1250,76561198081962345,1,1,Killing Floor,3.0
...,...,...,...,...,...,...
44734,367780,laislabonita75,2,1,Aero's Quest,5.0
44735,367780,evilindiegaming,2,1,Aero's Quest,5.0
44736,305920,laislabonita75,0,0,Another Perspective,0.0
44737,306040,Gamer0009,2,1,The Howler,5.0


In [61]:
new_df

,item_id,user_id,sentiment_analysis,recommend,app_name,rating
0,1250,76561197970982479,2,1,Killing Floor,5.0
1,1250,death-hunter,2,1,Killing Floor,5.0
2,1250,DJKamBer,0,1,Killing Floor,1.0
3,1250,diego9031,1,1,Killing Floor,3.0
4,1250,76561198081962345,1,1,Killing Floor,3.0
...,...,...,...,...,...,...
44734,367780,laislabonita75,2,1,Aero's Quest,5.0
44735,367780,evilindiegaming,2,1,Aero's Quest,5.0
44736,305920,laislabonita75,0,0,Another Perspective,0.0
44737,306040,Gamer0009,2,1,The Howler,5.0


In [46]:
reader = Reader(rating_scale=(0, 5))  # Adaptar rating_scale a tus datos

In [47]:
data = Dataset.load_from_df(df[['user_id', 'item_id', 'rating']], reader)

In [48]:
#Dividir el conjunto de datos en entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.2,random_state=42)  # Puedes ajustar el tamaño de prueba

In [15]:
#optimizacion de hiperparametros 
from surprise.model_selection import cross_validate

rmse_test_means = []
factores = [1,2,4,8,16,32,64,128,256]

for factor in factores:
    print(factor)
    model = SVD(n_factors=factor)
    cv = cross_validate(model, data, measures=['RMSE'], cv = 3, verbose=True)
    rmse_test_means.append(np.mean(cv['test_rmse']))

1
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.5519  1.5521  1.5608  1.5549  0.0042  
Fit time          0.16    0.15    0.24    0.18    0.04    
Test time         0.20    0.06    0.15    0.13    0.05    
2
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.5567  1.5556  1.5558  1.5560  0.0005  
Fit time          0.19    0.16    0.18    0.18    0.01    
Test time         0.08    0.16    0.08    0.11    0.03    
4
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.5567  1.5611  1.5539  1.5572  0.0030  
Fit time          0.23    0.17    0.21    0.20    0.03    
Test time         0.07    0.07    0.16    0.10    0.04    
8
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.5605  1.5

In [49]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_factors': [5,50,100],'n_epochs': [5, 10,20], 'lr_all': [0.001, 0.002, 0.005],
              'reg_all': [0.002, 0.02, 0.2]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, n_jobs = -1)
gs.fit(data)

In [50]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

1.5525539273094084
{'n_factors': 100, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.2}


In [19]:
best_model=gs.best_params

In [51]:
#Seleccionar un algoritmo (por ejemplo, SVD) y entrenar el modelo
model = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.2)
model.fit(trainset)

In [52]:
with open('modelo.pkl', 'wb') as archivo:
    pickle.dump(model, archivo)

In [53]:
with open ('modelo.pkl', 'rb') as archivo:
    modelo = pickle.load(archivo)

In [54]:
print(modelo)

In [66]:
def recomendacion_usuario( id_usuario ):
    if id_usuario not in new_df['user_id'].unique():
        return {'error': 'El usuario especificado no existe.'}
    #ID del usuario para el cual quieres obtener recomendaciones
    usuario_especifico = id_usuario  

    #Crear una lista de juegos ya valorados por el usuario específico
    juegos_valorados = new_df[new_df['user_id'] == usuario_especifico]['app_name'].unique()

    #Crear una lista de todos los juegos disponibles
    todos_los_juegos = new_df['app_name'].unique()

    #Crear una lista de juegos no valorados por el usuario específico
    juegos_no_valorados = list(set(todos_los_juegos) - set(juegos_valorados))

    #Generar predicciones para los juegos no valorados por el usuario
    predicciones = [modelo.predict(usuario_especifico, juego) for juego in juegos_no_valorados]

    #Ordenar las predicciones en base a la valoración y obtener los juegos recomendados
    recomendaciones = sorted(predicciones, key=lambda x: x.est, reverse=True)[:5]  # Obtener las 5 mejores recomendaciones

    #Mostrar los juegos recomendados
   
    lista=list()
    for recomendacion in recomendaciones:
        lista.append(recomendacion.iid)
    recomendaciones_dict = {
        'Juego1': lista[0],
        'Juego2': lista[1],
        'Juego3': lista[2],
        'Juego4': lista[3],
        'Juego5': lista[4]
    }
    return recomendaciones_dict
    

In [67]:
recomendacion_usuario('inorisanbaka')

{'Juego1': 'Cloudberry Kingdom™',
 'Juego2': 'Avaris 2: The Return of the Empress',
 'Juego3': 'Peggle Deluxe',
 'Juego4': 'LEGO® Worlds',
 'Juego5': 'SuperPower 2 Steam Edition'}

In [69]:
recomendacion_usuario("0")

{'error': 'El usuario especificado no existe.'}

In [70]:
recomendacion_usuario("")

{'error': 'El usuario especificado no existe.'}

In [71]:
recomendacion_usuario("inorisanbaka")

{'Juego1': 'Cloudberry Kingdom™',
 'Juego2': 'Avaris 2: The Return of the Empress',
 'Juego3': 'Peggle Deluxe',
 'Juego4': 'LEGO® Worlds',
 'Juego5': 'SuperPower 2 Steam Edition'}